# Anti-MXenes
## Here is the notebook to establish anti-MXene library from Materials Project database
### Part 1: Bulk materials in the form of AB2C2

In [ ]:
from pymatgen import Composition, MPRester

mpr = MPRester("YOUR_API_KEY")
anon_formula = {"A": 1, "B": 2, "C":2}
data = mpr.query({"anonymous_formula": anon_formula},
                             properties=["pretty_formula", "task_ids", "spacegroup"])

L_data = len(data)

myStructure0 = []
myStructure1 = []
myStructure2 = []
for i in range(L_data):
    formula = data[i]["pretty_formula"]
    if formula[-1] == "2" and formula[-2] == ")":
        myStructure0.append(data[i])
        
L0 = len(myStructure0)
for i in range(L0):
    spacegroup = myStructure0[i]["spacegroup"]
    if spacegroup["symbol"] == "I4/mmm" :
        myStructure1.append(myStructure0[i])

for i in range(L0):
    spacegroup = myStructure0[i]["spacegroup"]
    if spacegroup["symbol"] =="P-3m1":
        myStructure2.append(myStructure0[i])


L1 = len(myStructure1)
L2 = len(myStructure2)
f = open("01_ab2c2.dat","w")
f.write('AB2C2:  %d\n' %L_data)
f.write('A(BC)2:   %d\n' %L0)
f.write('A(BC)2 I4/mmm   %d \n' %L1)
f.write('A(BC)2 P-3m1   %d \n' %L2)
for i in range(L1):
    f.write(str(myStructure1[i]["pretty_formula"]))
    f.write("   ")
    f.write(str(myStructure1[i]["task_ids"]))
    f.write("\n")
f.close()

print(L_data, L0, L1)

The result shows that there are XXX bulk materials in the form of AB2C2, where XX materials is in the form of A(BC)2. And X materials are A(BC)2 with the symmetry of I4/mmm, which are parents of square-MXenes.

### Part 2: Analyze these parent materials based on maingroup elements. These main group elements include B, Al, Ga, C, Si, Ge, Sn, N, P, As, Sb, O, S, Se, Te

In [ ]:
content = []
with open('01_ab2c2.dat','r') as f:
    f.readline()
    f.readline()
    f.readline()
    f.readline()
    for line in f.readlines():
        content.append(line.split())

L = len(content)
print(L)
print(content[0][0][0]) #the first letter in formula

formula = {}
for key in ["B","C", "N", "P", "O", "S"]:
    element = []
    for i in range(L):
        for j in range(len(content[i][0])):
            if content[i][0][j] == key and not content[i][0][j+1].islower():
                element.append(content[i][0])
    formula[key] = element
for key in ["Al","Ga","Si","Ge","Sn", "As", "Sb", "Se", "Te"]:
    element = []
    for i in range(L):
        for j in range(len(content[i][0])):
            if content[i][0][j] == key[0] and content[i][0][j+1] == key[1]:
                element.append(content[i][0])
    formula[key] = element
#print(formula)

for key in formula.copy():
    if not formula[key]:
        formula.pop(key)
with open('02_elements.dat','w') as f:
    for key in formula:
        f.write(key)
        f.write('   ')
        f.write(str(len(formula[key])))
        f.write('\n')
        f.write(str(formula[key]))
        f.write('\n')

### Part 3: Let's deep into the composition to convenietly get the square-MXene sublayers.

In [ ]:
import re
def metalA(name, A):
    #print(len(name))
    if name[1].islower():
        A.append(name[0:2])
    else:
        A.append(name[0])
    return A

def metalBC(name, B, C):
    s1 = name.index('(')
    s2 = name.index(')')
    part = name[s1 + 1: s2]
    #print(part, key)
    m = re.search(key, part)
    m1 = m.span()[0]
    m2 = m.span()[1]
    C.append(part[m1:m2])
    if m1 > 0:
        B.append(part[0:m1])
    else:
        B.append(part[m2:])
    
    return B, C

content = []
with open('01_ab2c2.dat','r') as f:
    f.readline()
    f.readline()
    f.readline()
    f.readline()
    for line in f.readlines():
        content.append(line.split())

L = len(content)
#print(L)  # 623
#print(content[0][0][0]) #the first letter in formula

formula = {}
# find the C elements and the corresponding entry saved in list elements
for key in ["B","C", "N", "P", "O", "S"]: # elements with one character
    element = []
    for i in range(L):
        for j in range(len(content[i][0])):
            if content[i][0][j] == key and not content[i][0][j+1].islower():
                element.append(content[i][0])
    formula[key] = element
for key in ["Al","Ga","Si","Ge","Sn", "As", "Sb", "Se", "Te"]: # elements with two characters
    element = []
    for i in range(L):
        for j in range(len(content[i][0])):
            if content[i][0][j] == key[0] and content[i][0][j+1] == key[1]:
                element.append(content[i][0])
    formula[key] = element
#print(formula)

for key in formula.copy(): # some elements without entry should be deleted
    if not formula[key]:
        formula.pop(key)

for key in formula:
    tempDic = {}
    A, B, C = [], [], []
    L = len(formula[key])
    
    for i in range(L):
        name = formula[key][i]
        #print(content)
        #metal A
        metalA(name, A)
        #2D BC
        metalBC(name, B, C)

    tempDic['A'] = A
    tempDic['B'] = B
    tempDic['C'] = C
    #print(task_id, A, B, C)

    f = open('03/'+ key+'.dat', 'w')
    f.write('formula    ')
    f.write('A          ')
    f.write('B          ')
    f.write('C          ')
    f.write('\n')
    for i in range(len(A)):
        f.write(formula[key][i])
        f.write('  ')
        f.write(A[i])
        f.write('  ')
        f.write(B[i])
        f.write('  ')
        f.write(C[i])
        f.write('\n')
    f.close()

### Part 4: Retrive the cif files of parent bulks. 

In [10]:
from pymatgen import MPRester
import os

mpr = MPRester("YOUR_API_key")
elements = ["B", "N", "P", "O", "S", "Al","Ga","Si","Ge","Sn", "As", "Sb", "Se", "Te"]

for key in  elements:
    os.chdir(r'03')
    path = r'04/' + key
    os.mkdir(path)

    dataFind = []
    C, Ccopy = [], []
    f = open(key+'.dat','r')
    con1 = f.readlines()
    L1 = len(con1)
    
    for i in range(1, L1):
        C.append(con1[i].split())
    
    for i in range(len(C)):
        Ccopy.append(C[i][2])
    
    Ccopy = list(set(Ccopy))
    for i in Ccopy:
        for j in range(len(C)):
            if C[j][2] == i:
                dataFind.append(C[j])
                break

    for i in range(len(dataFind)):
        data = mpr.query({"pretty_formula": dataFind[i][0]},
                         properties=["spacegroup", "cif"])

        #print(i, dataFind[i][0],data)
        for j in range(len(data)):
                if data[j]['spacegroup']['symbol'] == 'I4/mmm':
                    tempDic = data[j]
                    break
        pretty_formula = dataFind[i][0]
        file = pretty_formula + '.cif'      
        os.chdir(path)
        f = open(file, 'w')
        f.write(tempDic['cif'])
        f.close()

### Part 5: We extract the 2D sublayers. First, convert all cif files to POSCAR files

In [13]:
from ase import io
vasppath = r'04'
os.chdir(vasppath)
elements = os.listdir(vasppath)
for element in elements:
    folderPath = vasppath + '/' + element
    os.chdir(folderPath)
    ciffiles = os.listdir()
    for cifs in ciffiles:
        atoms = io.read(cifs)
        atoms.write(cifs.split('.cif')[0]+'.vasp',format='vasp')
    os.chdir('..')


cp folder 04 to folder 05, only keep those POSCAR files and rename them to a proper name.

### Part 6: Extract 2D layers. You may find Ba(AsPd)2.vasp, Ce(PPt)2.vasp, is not the same to others. Exclude it and deal with it later.

In [ ]:
import os
targ_eles = ["B", "N", "P", "O", "S", "Al","Ga","Si","Ge","Sn", "As", "Sb", "Se", "Te"]
vasppath2 = r'05'
os.chdir(vasppath2)
elements = os.listdir(vasppath2)
# 1, find the element and the formula
for element in elements:
    folderPath2 = vasppath2 + '/' + element
    os.chdir(folderPath2)
    posfiles2 = os.listdir()
#     print(len(posfiles))
    newdir = r'06/' + element
    os.mkdir(newdir)
    for i in posfiles2:
        doc = folderPath2 + '/' + i
        print(i)        
        
        f = open(doc) # open the 3d POSCAR 
        lines = f.readlines() # contents in memory
        f.close()
        
        a = float(lines[2].split()[0])
        num = lines[5].split() # '1' has 3 conditions 
        formula = lines[0].split()
        if num.index('1') == 2:
            a1 = float(lines[7].split()[2])
            a2 = float(lines[8].split()[2])
            a3 = float(lines[9].split()[2])
            a4 = float(lines[10].split()[2])
            if formula[0] in targ_eles:
                nocenEle = formula[0]
                centerEle = formula[1]
            else:
                nocenEle = formula[1]
                centerEle = formula[0]
        elif num.index('1') == 1:
            a1 = float(lines[7].split()[2])
            a2 = float(lines[8].split()[2])
            a3 = float(lines[10].split()[2])
            a4 = float(lines[11].split()[2])
            if formula[0] in targ_eles:
                nocenEle = formula[0]
                centerEle = formula[2]
            else:
                nocenEle = formula[2]
                centerEle = formula[0]
        elif num.index('1') == 0:
            a1 = float(lines[8].split()[2])
            a2 = float(lines[9].split()[2])
            a3 = float(lines[10].split()[2])
            a4 = float(lines[11].split()[2])
            if formula[1] in targ_eles:
                nocenEle = formula[1]
                centerEle = formula[2]
            else:
                nocenEle = formula[2]
                centerEle = formula[1]
        
        tdchange = abs((a1 - a3)/2)
        clow = 0.5 - tdchange/20
        chigh = 0.5 + tdchange/20

        os.chdir(newdir)
        newfile = newdir + '/' + centerEle + nocenEle
        f = open(newfile, 'w')
        f.write(centerEle)
        f.write(' ')
        f.write(nocenEle)
        f.write('\n')
        f.write(str(1.000))
        f.write('\n')
        f.write('        ')
        f.write(str(a))
        f.write('        0.00000000        0.00000000')
        f.write('\n')
        f.write('        0.00000000        ')
        f.write(str(a))
        f.write('        0.00000000')
        f.write('\n')
        f.write('        0.00000000        0.00000000        20.00000000')
        f.write('\n')
        f.write('    ')
        f.write(centerEle)
        f.write('    ')
        f.write(nocenEle)
        f.write('\n')
        f.write('    ')
        f.write(str(2))
        f.write('    ')
        f.write(str(2))
        f.write('\n')
        f.write('Direct')
        f.write('\n')
        f.write('    0.500000000    0.000000000    0.50000000')
        f.write('\n')
        f.write('    0.000000000    0.500000000    0.50000000')
        f.write('\n')
        f.write('    0.500000000    0.500000000    ')
        f.write(str(clow))
        f.write('\n')
        f.write('    0.000000000    0.000000000    ')
        f.write(str(chigh))
        f.write('\n')
        f.close()
